In [1]:
!pip install torch
!pip install keras

In [96]:
import keras
from keras import layers
from keras.preprocessing import image
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import CSVLogger
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.applications.resnet import ResNet101
from keras.layers.convolutional import Conv2D, MaxPooling2D

In [34]:
INIT_LR = 1e-3
EPOCHS = 25
BS = 8

In [80]:
datagen = ImageDataGenerator(rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

In [104]:
train_data = datagen.flow_from_directory(
    directory = '/content/train',
    target_size = (224,224),
    class_mode='binary',
    shuffle=True
)

Found 80 images belonging to 2 classes.


In [105]:
val_data = datagen.flow_from_directory(
    directory = '/content/test',
    target_size = (224,224),
    class_mode='binary',
    shuffle=True
)

Found 20 images belonging to 2 classes.


In [106]:
val_data.class_indices

{'covid': 0, 'normal': 1}

In [89]:
#!rm -rf `find -type d -name .ipynb_checkpoints`

In [107]:
model = Sequential([
                    # BLOCK 1
                    Conv2D(32, kernel_size=(3,3), activation='relu', input_shape = (224, 224, 3)),
                    Conv2D(64, kernel_size=(3,3), activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Dropout(0.25),
                    # BLOCK 2
                    Conv2D(128, kernel_size=(3,3), activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Dropout(0.25),
                    # BLOCK 3
                    Conv2D(128, kernel_size=(3,3), activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Dropout(0.25),
                    # BLOCK 4
                    Conv2D(264, kernel_size=(3,3), activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Dropout(0.25),
                    # block 5
                    Flatten(),
                    Dense(64, activation='relu'),
                    Dropout(0.5),
                    Dense(1, activation='sigmoid')
])

In [108]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_195 (Conv2D)          (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_196 (Conv2D)          (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_197 (Conv2D)          (None, 108, 108, 128)     73856     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 54, 54, 128)       0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 54, 54, 128)      

In [114]:
history = model.fit(
    train_data,
    epochs = 5
)

Epoch 1/5
10/10 [==============================] - 29s 2s/step - loss: 0.2968 - accuracy: 0.9375


In [115]:
model.save('model.h5')

In [117]:
from keras.models import load_model

In [118]:
model = load_model('model.h5')

In [119]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_195 (Conv2D)          (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_196 (Conv2D)          (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_197 (Conv2D)          (None, 108, 108, 128)     73856     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 54, 54, 128)       0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 54, 54, 128)      

In [121]:
from PIL import Image

In [151]:
def predict(image_loc):
    sub = Image.open(image_loc)
    sub = sub.resize((224, 224))
    sub = np.array(sub)
    print(sub.shape)
    #sub = sub.reshape(-1, 224, 224, 3)
    sub = np.expand_dims(sub, axis=0)
    preds = model.predict(sub)
    #print(preds)
    return "normal" if preds else "covid" 
predict('/content/test/covid/covid21.jpeg')

(224, 224, 3)


'covid'

In [156]:
print("hello")

hello
